# [How to use chat models to call tools](https://python.langchain.com/docs/how_to/tool_calling/)

**Remember, while the name "tool calling" implies that the model is directly performing some action, this is actually not the case! The model only generates the arguments to a tool, and actually running the tool (or not) is up to the user.**

Tool calling is a general technique that generates structured output from a model, and you can use it even when you don't intend to invoke any tools

## Defining tool schemas
For a model to be able to call tools, we need to pass in tool schemas that describe what the tool does and what it's arguments are. Chat models that support tool calling features implement a .bind_tools() method for passing tool schemas to the model. Tool schemas can be passed in as Python functions (with typehints and docstrings), Pydantic models, TypedDict classes, or LangChain Tool objects. Subsequent invocations of the model will pass in these tool schemas along with the prompt.

### Python functions
Our tool schemas can be Python functions:


In [ ]:
# The function name, type hints, and docstring are all part of the tool
# schema that's passed to the model. Defining good, descriptive schemas
# is an extension of prompt engineering and is an important part of
# getting models to perform well.
def add(a: int, b: int) -> int:
    """Add two integers.

    Args:
        a: First integer
        b: Second integer
    """
    return a + b


def multiply(a: int, b: int) -> int:
    """Multiply two integers.

    Args:
        a: First integer
        b: Second integer
    """
    return a * b

### LangChain Tool
LangChain also implements a @tool decorator that allows for further control of the tool schema, such as tool names and argument descriptions. See the how-to guide here for details.

In [ ]:
from langchain_core.tools import tool

@tool
def add(a: int, b: int) -> int:
    """Add two integers.

    Args:
        a: First integer
        b: Second integer
    """
    return a + b

@tool
def multiply(a: int, b: int) -> int:
    """Multiply two integers.

    Args:
        a: First integer
        b: Second integer
    """
    return a * b

tools = [add, multiply]
tool_list = {"add": add, "multiply": multiply}

### Pydantic class
You can equivalently define the schemas without the accompanying functions using Pydantic.

Note that all fields are required unless provided a default value.

In [ ]:
from pydantic import BaseModel, Field


class add(BaseModel):
    """Add two integers."""

    a: int = Field(..., description="First integer")
    b: int = Field(..., description="Second integer")


class multiply(BaseModel):
    """Multiply two integers."""

    a: int = Field(..., description="First integer")
    b: int = Field(..., description="Second integer")

tools = [add, multiply]

### TypedDict class

In [ ]:
from typing_extensions import Annotated, TypedDict


class add(TypedDict):
    """Add two integers."""

    # Annotations must have the type and can optionally include a default value and description (in that order).
    a: Annotated[int, ..., "First integer"]
    b: Annotated[int, ..., "Second integer"]


class multiply(TypedDict):
    """Multiply two integers."""

    a: Annotated[int, ..., "First integer"]
    b: Annotated[int, ..., "Second integer"]


tools = [add, multiply]

To actually bind those schemas to a chat model, we'll use the `.bind_tools()` method. This handles converting the add and multiply schemas to the proper format for the model. The tool schema will then be passed it in each time the model is invoked.

In [ ]:
from dotenv import load_dotenv
load_dotenv()

from langchain_openai import ChatOpenAI
from pprint import pprint

llm = ChatOpenAI(model="gpt-4o-mini")

In [ ]:
llm_with_tools = llm.bind_tools(tools)

In [ ]:
query = "What is 3 * 12?"

tool_call = llm_with_tools.invoke(query)
pprint(tool_call)

As we can see our LLM generated arguments to a tool! You can look at the docs for bind_tools() to learn about all the ways to customize how your LLM selects tools, as well as [this](https://python.langchain.com/docs/how_to/tool_choice/) guide on how to force the LLM to call a tool rather than letting it decide.

## Tool calls
If tool calls are included in a LLM response, they are attached to the corresponding message or message chunk as a list of tool call objects in the `.tool_calls` attribute.

Note that chat models can call multiple tools at once.

A `ToolCall` is a typed dict that includes a tool name, dict of argument values, and (optionally) an identifier. Messages with no tool calls default to an empty list for this attribute.

In [ ]:
query = "What is 3 * 12? Also, what is 11 + 49?"

llm_with_tools.invoke(query).tool_calls

The `.tool_calls` attribute should contain valid tool calls. Note that on occasion, model providers may output malformed tool calls (e.g., arguments that are not valid JSON). When parsing fails in these cases, instances of InvalidToolCall are populated in the `.invalid_tool_calls` attribute. An `InvalidToolCall` can have a name, string arguments, identifier, and error message.

### Parsing
If desired, `output parsers` can further process the output. For example, we can convert existing values populated on the `.tool_calls` to Pydantic objects using the `PydanticToolsParser`:

In [ ]:
from langchain_core.output_parsers import PydanticToolsParser
from pydantic import BaseModel, Field
from langchain_core.tools import tool
from dotenv import load_dotenv
load_dotenv()

from langchain_openai import ChatOpenAI
from pprint import pprint

llm = ChatOpenAI(model="gpt-4o-mini")

llm_with_tools = llm.bind_tools(tools)

query = "What is 3 * 12? Also, what is 11 + 49?"
# query = "What is 3 * 12 + 2 ?"

class add(BaseModel):
    """Add two integers."""

    a: int = Field(..., description="First integer")
    b: int = Field(..., description="Second integer")


class multiply(BaseModel):
    """Multiply two integers."""

    a: int = Field(..., description="First integer")
    b: int = Field(..., description="Second integer")


tools = [add, multiply]
llm_with_tools = llm.bind_tools(tools)

chain = llm_with_tools | PydanticToolsParser(tools=tools)
res = chain.invoke(query)

In [ ]:
res